# ClinicsTestRunner
This is a Python port of the ClinicsTestRunner project, originally implemented in c# using LinqPad as the hosting environment

The data is processed with **py-linq** to show equivalence to the c# version,  
and also with **pandas** which is the more mainstream Python way.


## Imports
Libraries used by this notebook

In [1]:
from py_linq import Enumerable
from dateutil import parser
import datetime
import csv
import json
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import IntProgress, HTML, VBox
import ipywidgets as widgets


#### Notes on installing py_linq

**py-linq** is not available as a conda package, so 'conda install py-linq' fails.  
We can still use pip packages with conda, but need to install pip with conda first.

1. Open powershell in admin mode
2. Run 'conda install pip'
3. Run 'pip install py-linq'
---------------------

## Configuration and utility
Helper classes and data structures

##### Config

In [2]:
config = {
  "buildEnvironmentDSN": "build",
  "testEnvironmentDSN": "test1",
  "loadId": "Load 1.50",
  "extractFolder": "./data/",
  "rootFolder": "C:\Dev\Projects\migration-workbench\MigrationTestRunnerPython",
  "debugParser": False,
  "newBuildDSN": "dbInstance",
  "cutoverDate": parser.parse("2016-07-01")
}
config

{'buildEnvironmentDSN': 'build',
 'testEnvironmentDSN': 'test1',
 'loadId': 'Load 1.50',
 'extractFolder': './data/',
 'rootFolder': 'C:\\Dev\\Projects\\migration-workbench\\MigrationTestRunnerPython',
 'debugParser': False,
 'newBuildDSN': 'dbInstance',
 'cutoverDate': datetime.datetime(2016, 7, 1, 0, 0)}

#### Code mappings

In [3]:
status_map = {
  "P": "Booked",
  "X": "Cancelled",
  "N": "Not Attended",
  "T": "Reschedule",
  "H": "On Hold",
  "C": "Closed",
  "S": "Arrived Not Seen",
  "D": "Departed" 
}

type_map = {
  "N": "New",
  "R": "Review",
  "U": "Urgent"
}

active_map = {
  "A": "Active",
  "I": "Inactive"
}

#### Appointments class

In [4]:
class Appointment():

    @staticmethod
    def fromExtract(lineNo, r, cutoverDate = datetime.date.today()):
        a = Appointment()
        
        a.LineNo = lineNo
        
        a.Id = r[1]
#         a.Date = parser.parse(r[2].split()[0])   # Format guess is wrong for some lines (finds 612 Active instead of 640) 
        a.Date = datetime.datetime.strptime(r[2].split()[0], '%d/%m/%Y') 
        a.TimeSlot = r[3]
        a.Type = r[5][0:1]  #(N)ew, (U)rgent, (R)eview
        a.Specialty = r[6]
        a.Consultant = r[7]
        a.ClinicDesc = r[8]
        a.Status = r[9]
        a.FuturePast = "F" if a.Date > cutoverDate else "P"
        a.ActiveInactive = "A" if a.Date > cutoverDate and a.Status == "P" else "I"
        
        return a

    @staticmethod
    def fromCsv(r):
        a = Appointment()
        a.LineNo = r[0]
        a.Id = r[1]
        a.Date = parser.parse(r[2])
        a.TimeSlot = r[3]
        a.FuturePast = r[4]
        a.Type = r[5]
        a.Specialty = r[6]
        a.Consultant = r[7]
        a.ClinicDesc = r[8]
        a.Status = r[9]
        a.ActiveInactive = r[10]
        return a
    
    @staticmethod
    def fromJson(r):
        parsed = json.loads(r)
        a = Appointment()
        a.__dict__ = parsed
        a.Date = datetime.datetime.fromtimestamp(a.Date//1000).date()
        return a
    
    def __repr__(self):
        return str(self.__dict__)
    
    def __iter__(self):
        return iter([self.LineNo, self.Id, str(self.Date)[:10], self.TimeSlot, self.FuturePast, 
                     self.Type, self.Specialty, self.Consultant, self.ClinicDesc, 
                     self.Status, self.ActiveInactive])
    
    def to_dict(self):
        return {
            'LineNo':         self.LineNo,
            'Id':             self.Id,
            'Date':           self.Date,
            'TimeSlot':       self.TimeSlot,
            'FuturePast':     self.FuturePast,
            'Type':           self.Type,
            'Specialty':      self.Specialty,
            'Consultant':     self.Consultant,
            'ClinicDesc':     self.ClinicDesc,
            'Status':         self.Status,
            'ActiveInactive': self.ActiveInactive
        }


#### Progress bar

In [5]:
def log_progress(sequence, every=None, size=None, name='Items'):

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
#     label = HTML()
    label = widgets.Label(value='')
    box = VBox(children=[label, progress])
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

#### list_to_table

In [6]:
styles = """
    <style type='text/css'>
    table.app_table {
        border-collapse: collapse;
        border: 2px solid #17b;
        margin: 0.3em 0.2em;
    }

    td.app_td, th.app_th {
        text-align: left;
        vertical-align: top;
        border: 1px solid #aaa;
        margin: 0;
        padding: 0.3em;
    }

    th.app_th {
        background-color: #ddd;
        border: 1px solid #777;
        font-family: tahoma;
        font-size: 90%;
        font-weight: bold;
    }
    td.app_number, td.app_percent { 
        text-align: right
    }
    </style>
"""


def headersFromFirst(the_list):
    return list(the_list[0].keys())


def list_to_table(the_list, title=None): 
    html = []

    if (title):
        html.append(f'<h3>{title}</h3>') 
    
    html.append('<table class="app_table">') 

    headers = headersFromFirst(the_list)
    html.append('<tr class="app_tr">') 
    for header in headers:
        html.append(f'<th class="app_th">{header}</th>') 
    html.append('</tr>') 
    
    for row in the_list: 
        html.append('<tr class="app_tr">') 
        for cell in row:
            value = row[cell]
            if isinstance(value, list):
                value = list_to_table(value)
            elm_class = 'app_td'
            if isinstance(value, int):
                elm_class += ' app_number'
                value = '{:,}'.format(value)
            if isinstance(value, str) and value.endswith('%'):
                elm_class += ' app_percent'
            element = f'<td class="{elm_class}">{value}</td>'
            html.append(element)
        html.append('</tr>') 
    
    html.append('</table>') 
        
    return '\n'.join(html) + styles


## Transform extract data to appointment objects 

In [7]:
source = '{0}\data\AppointmentsExtract.txt'.format(config['rootFolder'])
target = '{0}\\temp\Appointments-{1}.csv'.format(config['rootFolder'], config['loadId']) 
with open(source, 'r') as extract, open(target, 'w') as appointment_csv:
    reader = csv.reader(extract, delimiter=',')
    writer = csv.writer(appointment_csv, lineterminator='\n')
    lineNo = 1
    logged_reader = log_progress(reader, size=10000, every=1000)
    for row in logged_reader:
        appointment = Appointment.fromExtract(lineNo, row, config['cutoverDate'])
        writer.writerow(list(appointment))
        lineNo += 1


## Status Code Sense Check
This report groups Appointments by Status then by FuturePast and ActiveInactive attributes.  
This breakdown is useful to get a sense of whether some appointments have been missed during extraction.


### Status Code Sense Check using py-linq

In [8]:
status_grouped = []

source = '{0}\\temp\Appointments-{1}.csv'.format(config['rootFolder'], config['loadId']) 
with open(source, 'r') as appointments_csv:
    reader = csv.reader(appointments_csv, delimiter=',')
    logged_reader = log_progress(reader, size=10000, every=100)
    status_grouped = Enumerable(logged_reader) \
        .select(lambda x: Appointment.fromCsv(x)) \
        .group_by(key_names=['Status'], key=lambda x: [x.Status]) \
        .select(lambda g: {
            'Status': status_map[g.key.Status], \
            'StatusCount': g.count(), \
            'StatusPercent': f'{g.count() / 100000 * 100:.2f}%', \
            'ByFuturePast': g.group_by( \
                key_names=['FuturePast', 'ActiveInactive'], \
                key=lambda x: [x.FuturePast, x.ActiveInactive]) \
             .select(lambda g: { \
                'FuturePast': 'Future' if g.key.FuturePast == 'F' else 'Past', \
                'ActiveInactive': 'Active' if g.key.ActiveInactive == 'A' else 'Inactive', \
                'Count': g.count(), \
                'Percent': f'{g.count() / 100000 * 100:.2f}%' \
              }) \
             .to_list()
          }) \
        .to_list()

html = list_to_table(status_grouped, 'Status Code Sense Check') 
display(HTML(html))
    

HTML(value='<h3>Status Code Sense Check</h3>\n<table class="app_table">\n<tr class="app_tr">\n<th class="app_t…

#### Format py-linq output as HTML table

### Status Code Sense Check using pandas

In [9]:
# Read the data into a dataframe
source = '{0}\\temp\Appointments-{1}.csv'.format(config['rootFolder'], config['loadId']) 
cols=['LineNo', 'Id', 'Date', 'Timeslot', 'FuturePast', 'Type', 'Specialty', 'Consultant', 'ClinicDesc', 'Status', 'ActiveInactive' ]
data = pd.read_csv(source, iterator=True, chunksize=1000, names=cols)
df = pd.concat(data, ignore_index=True)

totalCount = df['Id'].count()

def calcPercent(val):
    return f'{val / totalCount * 100:.2f}%' 

# Get inner grouping
byFuturePast = df.groupby(['Status', 'FuturePast', 'ActiveInactive']) \
    .size().to_frame(name = 'Count') \
    .reset_index()
byFuturePast['Percent'] = byFuturePast.Count.apply(calcPercent)
byFuturePast['Count'] = byFuturePast.Count.apply(lambda val: '{:,}'.format(val))

# Get outer grouping
byStatus = df.groupby('Status') \
    .size().to_frame(name = 'StatusCount') \
    .reset_index() 
byStatus['StatusPercent'] = byStatus.StatusCount.apply(calcPercent) 
byStatus['StatusCount'] = byStatus.StatusCount.apply(lambda val: '{:,}'.format(val)) 

# merge the two groups
merged = byStatus.merge(byFuturePast, how='outer')

# Change codes to descriptions
merged['Status'] = merged['Status'].apply(lambda val: status_map[val])
merged['FuturePast'] = merged['FuturePast'].apply(lambda val: 'Future' if val == 'F' else 'Past')
merged['ActiveInactive'] = merged['ActiveInactive'].apply(lambda val: 'Active' if val == 'A' else 'Inactive')

# Set index for better format
result = merged.set_index(['Status', 'StatusCount', 'StatusPercent', 'FuturePast', 'ActiveInactive'])
result.sort_values(by=['Status'], inplace=True)
result

Count  \
Status           StatusCount StatusPercent FuturePast ActiveInactive         
Arrived Not Seen 1,242       12.42%        Future     Inactive         593   
                                           Past       Inactive         649   
Booked           1,234       12.34%        Future     Active           640   
                                           Past       Inactive         594   
Cancelled        1,242       12.42%        Future     Inactive         615   
                                           Past       Inactive         627   
Closed           1,240       12.40%        Future     Inactive         656   
                                           Past       Inactive         584   
Departed         1,262       12.62%        Future     Inactive         622   
                                           Past       Inactive         640   
Not Attended     1,239       12.39%        Future     Inactive         610   
                                           Past       Inactive         629   
On Hold          1,230       12.30%        Future     Inactive         602   
                                           Past       Inactive         628   
Reschedule       1,311       13.11%        Future     Inactive         643   
                                           Past       Inactive         668   

                                                                     Percent  
Status           StatusCount StatusPercent FuturePast ActiveInactive          
Arrived Not Seen 1,242       12.42%        Future     Inactive         5.93%  
                                           Past       Inactive         6.49%  
Booked           1,234       12.34%        Future     Active           6.40%  
                                           Past       Inactive         5.94%  
Cancelled        1,242       12.42%        Future     Inactive         6.15%  
                                           Past       Inactive         6.27%  
Closed           1,240       12.40%        Future     Inactive         6.56%  
                                           Past       Inactive         5.84%  
Departed         1,262       12.62%        Future     Inactive         6.22%  
                                           Past       Inactive         6.40%  
Not Attended     1,239       12.39%        Future     Inactive         6.10%  
                                           Past       Inactive         6.29%  
On Hold          1,230       12.30%        Future     Inactive         6.02%  
                                           Past       Inactive         6.28%  
Reschedule       1,311       13.11%        Future     Inactive         6.43%  
                                           Past       Inactive         6.68%

## Match Appointments to Schedule
Check four match criteria for each appointment and attach a flag for each.  
- Use a left-join with the merge function and 'indicator=True'  
- Transform the result in the '\_merge' attribute to an appropriate value
- Add the results to the original dataframe as a new Series

In [10]:
appointments_file_path = '{0}\\temp\Appointments-{1}.csv'.format(config['rootFolder'], config['loadId']) 
cols=['LineNo', 'Id', 'Date', 'Timeslot', 'FuturePast', 'Type', 'Specialty', 'Consultant', 'ClinicDesc', 'Status', 'ActiveInactive' ]
df_appointments = pd.read_csv(appointments_file_path, names=cols, header=None)
df_appointments['Date'] = df_appointments['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

schedule_file_path = '{0}\\temp\clinicSchedule.json'.format(config['rootFolder'])
cols = ['Id', 'ClinicDesc', 'Specialty', 'Consultant', 'ClinicDate', 'TimeSlot']
df_schedule = pd.read_json(schedule_file_path)[cols]
# df_schedule.rename(columns={'ClinicDate': 'Date'}, inplace=True)
df_schedule['ClinicDate'] = df_schedule['ClinicDate'].apply(lambda x: datetime.datetime.strptime(x[0:10], '%Y-%m-%d'))

master_data_path = '{0}\\temp\masterData.json'.format(config['rootFolder'])
with open(master_data_path) as master_data_file:
    master_data = json.load(master_data_file)
df_attenders = pd.DataFrame(master_data['Attenders'])[['Specialty', 'Consultant']].drop_duplicates()
df_services = pd.DataFrame(master_data['Services'], columns=['Specialty', 'ConsultationType']).drop_duplicates()
df_services['ConsultationType'] = df_services['ConsultationType'].apply(lambda x: x[0])  # Take first char only 


df_appointments['AttenderMatch'] = \
    pd.merge(
        df_appointments, 
        df_attenders,  # de-duped above
        on = ['Specialty', 'Consultant'], 
        how = 'left', 
        indicator=True)['_merge'] \
    .apply(lambda val: 'NotMatched' if val == 'left_only' else 'Matched')
assert (df_appointments['Id'].count() == df_appointments['AttenderMatch'].count()), 'AttenderMatch has same count as Id'

df_appointments['ServiceMatch'] = \
    pd.merge(
        df_appointments, 
        df_services,  # de-duped above
        left_on = ['Specialty', 'Type'], 
        right_on = ['Specialty', 'ConsultationType'], 
        how = 'left', 
        indicator=True)['_merge'] \
    .apply(lambda val: 'NotMatched' if val == 'left_only' else 'Matched')
assert (df_appointments['Id'].count() == df_appointments['ServiceMatch'].count()), 'ServiceMatch has same count as Id'

# Adding a new Series correctly depends on it having exactly the same indexes as the original Dataframe
# Since df_schedule contains multiple occurances of the same Attender (due to multiplle clinic dates), 
# we need to de-dupe before performing the join otherwise some appointments will join n-times
schedule_attenders = df_schedule[['Specialty', 'Consultant']].drop_duplicates()
df_appointments['ScheduleExists'] = \
    pd.merge(
        df_appointments, 
        schedule_attenders, 
        on = ['Specialty', 'Consultant'],
        how = 'left', 
        indicator=True)['_merge'] \
    .apply(lambda val: 'No' if val == 'left_only' else 'Yes')
assert (df_appointments['Id'].count() == df_appointments['ScheduleExists'].count()), 'ScheduleExists has same count as Id'

schedule_dates = df_schedule[['Specialty', 'Consultant', 'ClinicDate']].drop_duplicates()
df_appointments['DateMatch'] = \
    pd.merge(
        df_appointments, 
        schedule_dates, 
        left_on = ['Specialty', 'Consultant', 'Date'], 
        right_on = ['Specialty', 'Consultant', 'ClinicDate'], 
        how = 'left', 
        indicator=True)['_merge'] \
    .apply(lambda val: 'NotMatched' if val == 'left_only' else 'Matched')
df_appointments.loc[df_appointments.ActiveInactive == 'I', 'DateMatch'] = 'Dont care'
assert (df_appointments['Id'].count() == df_appointments['DateMatch'].count()), 'DateMatch has same count as Id'

target = '{0}\\temp\\appointmentMatch.json'.format(config['rootFolder']) 
df_appointments.to_json(path_or_buf=target, orient='records', lines=True)
print('Done')

Done


## Unmatched Appoinments by Attender and Service
This section displays the match information from the above left-joins between Appointments and Clinic Schedules.  
We are mainly interested in details of the unmatched appointments, but also need to show the **% match attained** as a measure of progress.  

- Attenders (Specialty and Consultant)
- Service (Specialty and ConsultationType)
- Date is matched - for 'Active' appointments (Future and Booked) only 

### Functions for details display

These two functions encapsulate the code for grouping the data and calculating the information we want to highlight.  

##### 1. Aspects
We look at mismatches from two different 'aspects' - **Attender Match** and **Service Match**. The aspect is passed in to `get_breakdown_for(columns)` as the second column.  
- Attender match usually fails because a Consultant is not mapped correctly, or has a `DateTo` that preceeds the appointment.  
- Service match failures occur when the ConsultationType coding has changed but is not mapped. 
  

##### 2. Matched and Unmatched  
The first step is to separate matched and unmatched categories of the aspect.  
Since with matched records no further action is required, we can reduce the detail columns to the token value `<Matched>` and therefore reduce the volume of the report. The matched lines could be ommitted altogether, except that it is useful to see the percentage matched. This figure is reported to the Project in the weekly status report.

##### 3. Grouping function
To achieve the output line reduction for the matched records, we use a reduced level of grouping. Doing so produces `Nan` values for the columns we omit, and these are eventually replaced with the token value using `.fillna()`.

##### 4. Combining matched and unmatched groupings
This is done with a simple `.concat()` and is possible because we have performed `.reset_index()` on both groupings, so the two dataframes have the same index shape (i.e simple integer index). The resulting dataframe has the columns of the more complex grouping (the unmatched records).  

##### 5. Aggreagate function
The aggregate function `match_detail_aggreagates(x)` uses a dictionary to define the column names and the aggregation expression used to derive the column values. The data is returned by generating a series from the dictionary.


In [11]:
def match_detail_aggreagates(x):
    d = {}
    d['First_ClinicDesc'] = x['ClinicDesc'].iloc[0]
    d['Last_Date'] = x['Date'].max()
    d['CountDetail'] = x['Id'].count()
    return pd.Series(d, index=d.keys()) 


def get_breakdown_for(columns):
    aspect = columns[1]

    # Get grouping and details for 'NotMatched'
    notMatched = df_appointments[df_appointments[aspect] == 'NotMatched'] \
        .groupby(columns) \
        .apply(match_detail_aggreagates) \
        .reset_index()

    # Ensure an empty dataframe if no results are 'NotMatched'
    if(notMatched.size == 0):
        notMatched = pd.DataFrame(columns=columns)

    # Get grouping and details for 'Matched'
    matched = df_appointments[df_appointments[aspect] == 'Matched'] \
        .groupby(columns[0:2] + ['DateMatch']) \
        .apply(match_detail_aggreagates) \
        .reset_index()

    # Ensure an empty dataframe if no results are 'Matched'
    if(matched.size == 0):
        matched = pd.DataFrame(columns=columns)

    # Create a combined dataframe
    details = pd.concat([notMatched, matched], sort=True)

    # Fill out detail columns for matched that we don't care about 
    details.fillna('<Matched>', inplace=True)

    # Set column order
    details = details[columns + ['First_ClinicDesc', 'Last_Date', 'CountDetail']]

    return details


### Attender Match  - unmatched appointments

In [12]:
# Get attender match details
attenderDetails = get_breakdown_for(['ActiveInactive', 'AttenderMatch', 'Specialty', 'Consultant', 'DateMatch'])

# Count higher level groupings and merge with details
byActiveInactive = df_appointments.groupby(['ActiveInactive']) \
    .size().to_frame(name='ActiveCount').reset_index()

byAttenderMatch = df_appointments.groupby(['ActiveInactive', 'AttenderMatch']) \
    .size().to_frame(name='AttenderMatchCount').reset_index() 

attenders = byActiveInactive \
    .merge(byAttenderMatch, on='ActiveInactive', how='left') \
    .merge(attenderDetails, on=['ActiveInactive', 'AttenderMatch'], how='left')

# Adjust display values
attenders['Active'] = attenders['ActiveInactive'].apply(lambda x: active_map[x])
attenders['Match%'] = attenders.apply(lambda x: f'{(x.AttenderMatchCount / x.ActiveCount * 100):.2f}%', axis=1 )
attenders['Count'] = attenders['AttenderMatchCount'].apply(lambda val: '{:,}'.format(val))
attenders['ActiveCount'] = attenders['ActiveCount'].apply(lambda val: '{:,}'.format(val))

# Order columns
attenders_columns = ['Active', 'ActiveCount', 'AttenderMatch', 'Count', 'Match%', 'Specialty', 'Consultant', 'DateMatch', 'First_ClinicDesc', 'Last_Date', 'CountDetail']
attenders = attenders[attenders_columns]

# Shape the list to stagger groupings to the left
attenders = attenders.set_index(attenders_columns[0:7])

pd.options.display.max_rows = 999
attenders



DateMatch  \
Active   ActiveCount AttenderMatch Count Match% Specialty Consultant               
Active   640         Matched       530   82.81% <Matched> <Matched>      Matched   
                                                          <Matched>   NotMatched   
                     NotMatched    110   17.19% DERM      Cons12      NotMatched   
                                                          Cons26      NotMatched   
                                                          Cons34      NotMatched   
                                                          Cons71      NotMatched   
                                                          Cons79      NotMatched   
                                                          Cons99      NotMatched   
                                                EU        Cons88      NotMatched   
                                                NEURO     Cons06      NotMatched   
                                                          Cons17      NotMatched   
                                                ORTHO     Cons06      NotMatched   
                                                          Cons32      NotMatched   
                                                          Cons35      NotMatched   
                                                          Cons76      NotMatched   
                                                          Cons84      NotMatched   
                                                          Cons88      NotMatched   
                                                          Cons92      NotMatched   
Inactive 9,360       Matched       7,850 83.87% <Matched> <Matched>    Dont care   
                     NotMatched    1,510 16.13% DERM      Cons12       Dont care   
                                                          Cons26       Dont care   
                                                          Cons34       Dont care   
                                                          Cons71       Dont care   
                                                          Cons79       Dont care   
                                                          Cons99       Dont care   
                                                EU        Cons88       Dont care   
                                                NEURO     Cons06       Dont care   
                                                          Cons17       Dont care   
                                                ORTHO     Cons06       Dont care   
                                                          Cons32       Dont care   
                                                          Cons35       Dont care   
                                                          Cons76       Dont care   
                                                          Cons84       Dont care   
                                                          Cons88       Dont care   
                                                          Cons92       Dont care   

                                                                                      First_ClinicDesc  \
Active   ActiveCount AttenderMatch Count Match% Specialty Consultant                                     
Active   640         Matched       530   82.81% <Matched> <Matched>         Cons61 EU clinic on Friday   
                                                          <Matched>      Cons32 DERM clinic on Tuesday   
                     NotMatched    110   17.19% DERM      Cons12        Cons12 DERM clinic on Thursday   
                                                          Cons26         Cons26 DERM clinic on Tuesday   
                                                          Cons34          Cons34 DERM clinic on Friday   
                                                          Cons71          Cons71 DERM clinic on Sunday   
                                                          Cons79          Cons79 DERM clinic on Sunday   
        

### Service Match  - unmatched appointments

In [13]:
# Get service match details
serviceDetails = get_breakdown_for(['ActiveInactive', 'ServiceMatch', 'Specialty', 'Type', 'DateMatch'])
serviceDetails['Type'] = serviceDetails['Type'].apply(lambda x: '<Matched>' if x == '<Matched>' else type_map[x])

# Count higher level groupings and merge with details
byActiveInactive = df_appointments.groupby(['ActiveInactive']) \
    .size().to_frame(name='ActiveCount').reset_index()

byServiceMatch = df_appointments.groupby(['ActiveInactive', 'ServiceMatch']) \
    .size().to_frame(name='ServiceMatchCount').reset_index() 

services = byActiveInactive \
    .merge(byServiceMatch, on='ActiveInactive', how='left') \
    .merge(serviceDetails, on=['ActiveInactive', 'ServiceMatch'], how='left')

# Adjust display values
services['Active'] = services['ActiveInactive'].apply(lambda x: active_map[x])
services['Match%'] = services.apply(lambda x: f'{(x.ServiceMatchCount / x.ActiveCount * 100):.2f}%', axis=1 )
services['Count'] = services['ServiceMatchCount'].apply(lambda val: '{:,}'.format(val))
services['ActiveCount'] = services['ActiveCount'].apply(lambda val: '{:,}'.format(val))

# Order columns
services_cols = ['Active', 'ActiveCount', 'ServiceMatch', 'Count', 'Match%', 'Specialty', 'Type', 'DateMatch', 'First_ClinicDesc', 'Last_Date', 'CountDetail']
services = services[services_cols]

# Shape the list to staggered-left
services = services.set_index(services_cols[0:7])

pd.options.display.max_rows = 999
services



DateMatch  \
Active   ActiveCount ServiceMatch Count Match% Specialty Type                    
Active   640         Matched      507   79.22% <Matched> <Matched>     Matched   
                                                         <Matched>  NotMatched   
                     NotMatched   133   20.78% EU        New        NotMatched   
                                                         Review        Matched   
                                                         Review     NotMatched   
Inactive 9,360       Matched      7,485 79.97% <Matched> <Matched>   Dont care   
                     NotMatched   1,875 20.03% EU        New         Dont care   
                                                         Review      Dont care   

                                                                                 First_ClinicDesc  \
Active   ActiveCount ServiceMatch Count Match% Specialty Type                                       
Active   640         Matched      507   79.22% <Matched> <Matched>     Cons61 EU clinic on Friday   
                                                         <Matched>  Cons32 DERM clinic on Tuesday   
                     NotMatched   133   20.78% EU        New           Cons88 EU clinic on Monday   
                                                         Review     Cons04 EU clinic on Wednesday   
                                                         Review     Cons08 EU clinic on Wednesday   
Inactive 9,360       Matched      7,485 79.97% <Matched> <Matched>   Cons71 DERM clinic on Friday   
                     NotMatched   1,875 20.03% EU        New         Cons81 EU clinic on Thursday   
                                                         Review      Cons45 EU clinic on Thursday   

                                                                    Last_Date  \
Active   ActiveCount ServiceMatch Count Match% Specialty Type                   
Active   640         Matched      507   79.22% <Matched> <Matched> 2016-12-25   
                                                         <Matched> 2016-12-30   
                     NotMatched   133   20.78% EU        New       2016-12-29   
                                                         Review    2016-12-13   
                                                         Review    2016-12-26   
Inactive 9,360       Matched      7,485 79.97% <Matched> <Matched> 2016-12-30   
                     NotMatched   1,875 20.03% EU        New       2016-12-30   
                                                         Review    2016-12-30   

                                                                    CountDetail  
Active   ActiveCount ServiceMatch Count Match% Specialty Type                    
Active   640         Matched      507   79.22% <Matched> <Matched>           21  
                                                         <Matched>          486  
                     NotMatched   133   20.78% EU        New                 65  
                                                         Review               4  
                                                         Review              64  
Inactive 9,360       Matched      7,485 79.97% <Matched> <Matched>         7485  
                     NotMatched   1,875 20.03% EU        New                941  
                                                         Review             934

## Clinic Description Match

In [14]:
def get_aggreagates2(x):
    d = {}
    d['Last_Date'] = x['Date'].max()
    d['Count'] = x['Id'].count()
    return pd.Series(d, index=['Last_Date', 'Count']) 

def normalized(s):
    s2 = s.upper().strip()
    s2 = ''.join(c for c in s2 if c.isalnum())
    return s2


# Get unique Clinic Descriptiosn from appointments for comparison to Schedule 
df_scheduleGroup = df_appointments[df_appointments['ActiveInactive'] == 'A'] \
    .groupby(['Specialty', 'Consultant', 'ClinicDesc']) \
    .apply(get_aggreagates2) \
    .reset_index()
df_scheduleGroup['ClinicDescNormalized'] = df_scheduleGroup['ClinicDesc'].apply(normalized)

# Normalize the schedule Clinic Descriptions
df_schedule2 = df_schedule['ClinicDesc'].apply(normalized).to_frame()
df_schedule2.rename(columns={'ClinicDesc':'ClinicDescNormalized'}, inplace=True)
df_schedule2.drop_duplicates(inplace=True)

# Compare
df_scheduleGroup['ScheduleExists'] = \
    pd.merge(
        df_scheduleGroup, 
        df_schedule2,
        on = ['ClinicDescNormalized'], 
        how = 'left', 
        indicator=True)['_merge'] \
    .apply(lambda val: 'No' if val == 'left_only' else 'Yes')

# Create copy and rename columns with _code suffix
noClinic = df_scheduleGroup[df_scheduleGroup['ScheduleExists'] == 'No'].copy()
noClinic.rename(columns={'Specialty':'Specialty_Code', 'Consultant':'Consultant_Code'}, inplace=True)


# Get Specialty descriptions and termination dates
df_specialties = pd.DataFrame(master_data['Specialties'])[['Code', 'Description', 'DateTo']].drop_duplicates()
noClinic[['Specialty', 'Specialty_DateTo']] = \
    noClinic.join(          # pd.merge() joins exactly one RHS to each LHS, so duplicates on LHS fail the join (return NaN)
        df_specialties.set_index('Code'),  # right_on='Code'
        on='Specialty_Code',
        how = 'left', 
        ) \
    [['Description', 'DateTo']]


# Get Consultant descriptions and termination dates
df_consultants = pd.DataFrame(master_data['Consultants'])[['Code', 'Description', 'DateTo']].drop_duplicates()
noClinic[['Consultant', 'Consultant_DateTo']] = \
    noClinic.join(          # pd.merge() joins exactly one RHS to each LHS, so duplicates on LHS fail the join (return NaN)
        df_consultants.set_index('Code'),  # right_on='Code'
        on='Consultant_Code',
        how = 'left', 
        ) \
    [['Description', 'DateTo']]


# Set columns to display
noClinic = noClinic[['ClinicDesc', 'Specialty', 'Specialty_DateTo', 'Consultant', 'Consultant_DateTo', 'Count', 'Last_Date']]

# Order by ClinicDesc
noClinic.set_index('ClinicDesc', inplace=True)
noClinic.sort_values('ClinicDesc', inplace=True)

noClinic



,Specialty,Specialty_DateTo,Consultant,Consultant_DateTo,Count,Last_Date
ClinicDesc,,,,,,
Cons05 NEURO clinic on Friday,Neurology,2020-12-20,Consultant05,2016-08-29,1,2016-12-27
Cons05 NEURO clinic on Monday,Neurology,2020-12-20,Consultant05,2016-08-29,1,2016-09-06
Cons06 NEURO clinic on Friday,Neurology,2020-12-20,Consultant06,2016-09-11,1,2016-08-12
Cons06 NEURO clinic on Monday,Neurology,2020-12-20,Consultant06,2016-09-11,1,2016-10-24
Cons06 NEURO clinic on Saturday,Neurology,2020-12-20,Consultant06,2016-09-11,1,2016-07-28
Cons06 NEURO clinic on Sunday,Neurology,2020-12-20,Consultant06,2016-09-11,1,2016-12-23
Cons06 NEURO clinic on Tuesday,Neurology,2020-12-20,Consultant06,2016-09-11,2,2016-12-04
Cons06 NEURO clinic on Wednesday,Neurology,2020-12-20,Consultant06,2016-09-11,1,2016-10-22
Cons06 ORTHO clinic on Friday,Orthopaedics,2020-12-20,Consultant06,2016-09-11,2,2016-09-15


## Schedules and Dates

### Appointments (Active) where Attender has no Schedule

#### ...using pandas

In [15]:
def noSchedule_aggreagates(x):
    d = {}
    d['Last_Date'] = x['Date'].max()
    d['Count'] = x['Id'].count()
    return pd.Series(d, index=['Last_Date', 'Count']) 


noSchedule = df_appointments[(df_appointments['ActiveInactive'] == 'A') & (df_appointments['ScheduleExists'] == 'No')] \
    .groupby(['Specialty', 'Consultant', 'ClinicDesc']) \
    .apply(noSchedule_aggreagates) \
    .reset_index()

noSchedule


,Specialty,Consultant,ClinicDesc,Last_Date,Count
0,DERM,Cons12,Cons12 DERM clinic on Friday,2016-08-26,2
1,DERM,Cons12,Cons12 DERM clinic on Monday,2016-08-04,1
2,DERM,Cons12,Cons12 DERM clinic on Sunday,2016-11-10,1
3,DERM,Cons12,Cons12 DERM clinic on Thursday,2016-09-16,2
4,DERM,Cons12,Cons12 DERM clinic on Tuesday,2016-11-15,2
5,DERM,Cons12,Cons12 DERM clinic on Wednesday,2016-08-01,1
6,DERM,Cons26,Cons26 DERM clinic on Monday,2016-11-07,2
7,DERM,Cons26,Cons26 DERM clinic on Saturday,2016-10-22,1
8,DERM,Cons26,Cons26 DERM clinic on Sunday,2016-08-13,1
9,DERM,Cons26,Cons26 DERM clinic on Tuesday,2016-07-25,1


#### ...using py-linq

In [16]:
source = '{0}\\temp\\appointmentMatch.json'.format(config['rootFolder']) 
with open(source, 'r') as appointmentMatch_json:
    appointments = Enumerable(appointmentMatch_json) \
        .select(lambda x: Appointment.fromJson(x)) \
        .where(lambda x: x.ActiveInactive == 'A') \
        .group_by(
            key_names=['Specialty', 'Consultant', 'ClinicDesc'], 
            key=lambda x: [x.Specialty, x.Consultant, x.ClinicDesc]
        ) \
        .select(lambda g: {
            'Specialty_Code': g.key.Specialty,
            'Consultant_Code': g.key.Consultant,
            'ClinicDesc': g.key.ClinicDesc,
            'Count': g.count(),
            'LastDate': max(appointment.Date for appointment in g),
            'ScheduleExists': g.first().ScheduleExists
          }) 

master_data_path = '{0}\\temp\masterData.json'.format(config['rootFolder'])
with open(master_data_path) as master_data_file:
    master_data = json.load(master_data_file)
    specialties = master_data['Specialties']
    consultants = master_data['Consultants']
    

noSchedule = appointments \
    .where(lambda x: x['ScheduleExists'] == 'No') 

noSchedule = noSchedule \
    .join( \
        Enumerable(specialties),
        outer_key=lambda x: x['Specialty_Code'],
        inner_key=lambda x: x['Code'],
        result_func=lambda x: x
     ) \
    .default_if_empty() \
    .select(lambda j: {
        'Specialty_Code': j[0]['Specialty_Code'],
        'Specialty': j[1]['Description'],
        'Specialty_ToDate': j[1]['DateTo'],
        'Consultant_Code': j[0]['Consultant_Code'],
        'ClinicDesc': j[0]['ClinicDesc'],
        'Count': j[0]['Count'],
        'LastDate': j[0]['LastDate']
      }) 

noSchedule = noSchedule \
    .join( \
        Enumerable(consultants),
        outer_key=lambda x: x['Consultant_Code'],
        inner_key=lambda x: x['Code'],
        result_func=lambda x: x
     ) \
    .default_if_empty() \
    .select(lambda j: {
        'Specialty Code': j[0]['Specialty_Code'], 
        'Specialty': j[0]['Specialty'], 
        'Specialty ToDate': j[0]['Specialty_ToDate'], 
        'Consultant Code': j[0]['Consultant_Code'], 
        'Consultant': j[1]['Description'], 
        'Consultant DateTo': j[1]['DateTo'], 
        'ClinicDesc': j[0]['ClinicDesc'], 
        'Count': j[0]['Count'], 
        'LastDate': j[0]['LastDate']
      })

noSchedule = noSchedule.to_list()

html = list_to_table(noSchedule, 'Appointments (Active) where Attender has no Schedule') 
display(HTML(html))


HTML(value='<h3>Appointments (Active) where Attender has no Schedule</h3>\n<table class="app_table">\n<tr clas…

### Appointments (Active) not matching on Clinic Date

#### ...using pandas

In [17]:
appointments_noDateMatch = df_appointments[
    (df_appointments['ActiveInactive'] == 'A') & \
    (df_appointments['ScheduleExists'] == 'Yes') & \
    (df_appointments['DateMatch'] == 'NotMatched')] \
    [['Specialty', 'Consultant', 'Id', 'Date']]

noDateMatch = appointments_noDateMatch \
    .groupby(['Specialty', 'Consultant']) \
    .size().to_frame(name='Count') \
    .reset_index() \
    .merge(appointments_noDateMatch, on=['Specialty', 'Consultant'], how='left') \
    .set_index(['Specialty', 'Consultant', 'Count', 'Id', 'Date'])

noDateMatch


Empty DataFrame
Columns: []
Index: [(DERM, Cons05, 6, Id522, 2016-10-13 00:00:00), (DERM, Cons05, 6, Id735, 2016-10-25 00:00:00), (DERM, Cons05, 6, Id937, 2016-07-08 00:00:00), (DERM, Cons05, 6, Id1914, 2016-08-03 00:00:00), (DERM, Cons05, 6, Id3006, 2016-07-30 00:00:00), (DERM, Cons05, 6, Id8819, 2016-11-18 00:00:00), (DERM, Cons100, 5, Id6672, 2016-11-25 00:00:00), (DERM, Cons100, 5, Id6714, 2016-11-29 00:00:00), (DERM, Cons100, 5, Id7073, 2016-12-10 00:00:00), (DERM, Cons100, 5, Id7463, 2016-09-12 00:00:00), (DERM, Cons100, 5, Id8743, 2016-12-25 00:00:00), (DERM, Cons14, 6, Id2012, 2016-07-26 00:00:00), (DERM, Cons14, 6, Id5239, 2016-11-19 00:00:00), (DERM, Cons14, 6, Id7099, 2016-10-06 00:00:00), (DERM, Cons14, 6, Id9004, 2016-07-26 00:00:00), (DERM, Cons14, 6, Id9398, 2016-10-20 00:00:00), (DERM, Cons14, 6, Id9945, 2016-09-16 00:00:00), (DERM, Cons25, 8, Id974, 2016-08-08 00:00:00), (DERM, Cons25, 8, Id1147, 2016-08-03 00:00:00), (DERM, Cons25, 8, Id5661, 2016-12-18 00:00:00), (DERM, Cons25, 8, Id7001, 2016-09-15 00:00:00), (DERM, Cons25, 8, Id7769, 2016-07-03 00:00:00), (DERM, Cons25, 8, Id9014, 2016-10-26 00:00:00), (DERM, Cons25, 8, Id9223, 2016-07-07 00:00:00), (DERM, Cons25, 8, Id9888, 2016-09-02 00:00:00), (DERM, Cons31, 7, Id110, 2016-12-11 00:00:00), (DERM, Cons31, 7, Id2157, 2016-11-25 00:00:00), (DERM, Cons31, 7, Id4759, 2016-08-13 00:00:00), (DERM, Cons31, 7, Id7402, 2016-07-20 00:00:00), (DERM, Cons31, 7, Id7542, 2016-09-24 00:00:00), (DERM, Cons31, 7, Id7581, 2016-08-20 00:00:00), (DERM, Cons31, 7, Id9195, 2016-08-14 00:00:00), (DERM, Cons32, 4, Id15, 2016-07-28 00:00:00), (DERM, Cons32, 4, Id5209, 2016-10-08 00:00:00), (DERM, Cons32, 4, Id5998, 2016-10-30 00:00:00), (DERM, Cons32, 4, Id8392, 2016-07-11 00:00:00), (DERM, Cons33, 5, Id2599, 2016-12-29 00:00:00), (DERM, Cons33, 5, Id3122, 2016-09-21 00:00:00), (DERM, Cons33, 5, Id5356, 2016-07-29 00:00:00), (DERM, Cons33, 5, Id6404, 2016-10-23 00:00:00), (DERM, Cons33, 5, Id8892, 2016-12-23 00:00:00), (DERM, Cons43, 7, Id1118, 2016-07-08 00:00:00), (DERM, Cons43, 7, Id1649, 2016-11-04 00:00:00), (DERM, Cons43, 7, Id4223, 2016-12-06 00:00:00), (DERM, Cons43, 7, Id4387, 2016-08-22 00:00:00), (DERM, Cons43, 7, Id5861, 2016-08-28 00:00:00), (DERM, Cons43, 7, Id7135, 2016-09-24 00:00:00), (DERM, Cons43, 7, Id8418, 2016-11-15 00:00:00), (DERM, Cons48, 5, Id1961, 2016-08-17 00:00:00), (DERM, Cons48, 5, Id2436, 2016-12-18 00:00:00), (DERM, Cons48, 5, Id5332, 2016-10-28 00:00:00), (DERM, Cons48, 5, Id7950, 2016-08-30 00:00:00), (DERM, Cons48, 5, Id9944, 2016-09-14 00:00:00), (DERM, Cons54, 8, Id3484, 2016-07-27 00:00:00), (DERM, Cons54, 8, Id3667, 2016-08-01 00:00:00), (DERM, Cons54, 8, Id4120, 2016-07-27 00:00:00), (DERM, Cons54, 8, Id6906, 2016-11-10 00:00:00), (DERM, Cons54, 8, Id7366, 2016-09-28 00:00:00), (DERM, Cons54, 8, Id8261, 2016-07-18 00:00:00), (DERM, Cons54, 8, Id9275, 2016-12-14 00:00:00), (DERM, Cons54, 8, Id9538, 2016-11-21 00:00:00), (DERM, Cons74, 6, Id213, 2016-10-28 00:00:00), (DERM, Cons74, 6, Id1763, 2016-09-09 00:00:00), (DERM, Cons74, 6, Id6508, 2016-08-07 00:00:00), (DERM, Cons74, 6, Id6914, 2016-10-26 00:00:00), (DERM, Cons74, 6, Id8377, 2016-10-03 00:00:00), (DERM, Cons74, 6, Id8924, 2016-10-21 00:00:00), (DERM, Cons78, 7, Id130, 2016-07-02 00:00:00), (DERM, Cons78, 7, Id2419, 2016-09-25 00:00:00), (DERM, Cons78, 7, Id3088, 2016-11-14 00:00:00), (DERM, Cons78, 7, Id3590, 2016-07-11 00:00:00), (DERM, Cons78, 7, Id6181, 2016-09-05 00:00:00), (DERM, Cons78, 7, Id7120, 2016-07-12 00:00:00), (DERM, Cons78, 7, Id9880, 2016-11-29 00:00:00), (DERM, Cons84, 3, Id505, 2016-12-20 00:00:00), (DERM, Cons84, 3, Id1957, 2016-11-03 00:00:00), (DERM, Cons84, 3, Id3194, 2016-12-23 00:00:00), (DERM, Cons87, 7, Id1127, 2016-09-22 00:00:00), (DERM, Cons87, 7, Id2176, 2016-12-29 00:00:00), (DERM, Cons87, 7, Id3529, 2016-12-29 00:00:00), (DERM, Cons87, 7, Id7987, 2016-08-31 00:00:00), (DERM, Cons87, 7, Id8168, 2016-08-27 00:00:00), (DERM, Cons87, 7, Id8729, 2016-12

#### ...using py-linq

In [18]:
source = '{0}\\temp\\appointmentMatch.json'.format(config['rootFolder']) 
with open(source, 'r') as appointmentMatch_json:
    appointments = Enumerable(appointmentMatch_json) \
        .select(lambda x: Appointment.fromJson(x)) \
        .where(lambda x: x.ActiveInactive == 'A') \
        .group_by(
            key_names=['Specialty', 'Consultant', 'ClinicDesc'], 
            key=lambda x: [x.Specialty, x.Consultant, x.ClinicDesc]
        ) \
        .select(lambda g: {
            'Specialty_Code': g.key.Specialty,
            'Consultant_Code': g.key.Consultant,
            'ClinicDesc': g.key.ClinicDesc,
            'Count': g.count(),
            'LastDate': max(appointment.Date for appointment in g),
            'ScheduleExists': g.first().ScheduleExists
          }) 

master_data_path = '{0}\\temp\masterData.json'.format(config['rootFolder'])
with open(master_data_path) as master_data_file:
    master_data = json.load(master_data_file)
    specialties = master_data['Specialties']
    consultants = master_data['Consultants']
    

noSchedule = appointments \
    .where(lambda x: x['ScheduleExists'] == 'No') 

noSchedule = noSchedule \
    .join( \
        Enumerable(specialties),
        outer_key=lambda x: x['Specialty_Code'],
        inner_key=lambda x: x['Code'],
        result_func=lambda x: x
     ) \
    .default_if_empty() \
    .select(lambda j: {
        'Specialty_Code': j[0]['Specialty_Code'],
        'Specialty': j[1]['Description'],
        'Specialty_ToDate': j[1]['DateTo'],
        'Consultant_Code': j[0]['Consultant_Code'],
        'ClinicDesc': j[0]['ClinicDesc'],
        'Count': j[0]['Count'],
        'LastDate': j[0]['LastDate']
      }) 

noSchedule = noSchedule \
    .join( \
        Enumerable(consultants),
        outer_key=lambda x: x['Consultant_Code'],
        inner_key=lambda x: x['Code'],
        result_func=lambda x: x
     ) \
    .default_if_empty() \
    .select(lambda j: {
        'Specialty Code': j[0]['Specialty_Code'], 
        'Specialty': j[0]['Specialty'], 
        'Specialty ToDate': j[0]['Specialty_ToDate'], 
        'Consultant Code': j[0]['Consultant_Code'], 
        'Consultant': j[1]['Description'], 
        'Consultant DateTo': j[1]['DateTo'], 
        'ClinicDesc': j[0]['ClinicDesc'], 
        'Count': j[0]['Count'], 
        'LastDate': j[0]['LastDate']
      })

noSchedule = noSchedule.to_list()

html = list_to_table(noSchedule, 'Appointments (Active) not matching on Clinic Date') 
display(HTML(html))


HTML(value='<h3>Appointments (Active) not matching on Clinic Date</h3>\n<table class="app_table">\n<tr class="…